## Import Modules

In [1]:
# Set paths
import os
from imp import reload

# Data manipulation
import pandas as pd
import numpy as np
from scipy import stats

# Custom package for data preprocessing
import mytools as mt 

# Set notebook options
pd.set_option("display.max_columns", 150)
pd.set_option("display.max_rows", 10000)
pd.set_option("display.float_format", lambda x: '%.2f' % x)

# Pretty display of multiple functions in a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### File Location

In [2]:
# Data paths
import filepaths

## Load Data

In [3]:
df_transactions = pd.read_csv(filepaths.interim_transactions_data, sep=',')

In [5]:
df_transactions.head(2)

,trans_id,customer_id,trans_date,prod_subcat_code,prod_cat_code,qty,rate,tax,total_amt,store_type,returned_item,drop_record,prod_cat,prod_subcat,counter,assessment_date,duration,trans_date.day,trans_date.month_num,trans_date.year,trans_date.year_month,trans_date.hour,trans_date.weekday,trans_date.week_of_year,trans_date.month,trans_date.weekday_num,trans_date.week_of_month,first_purchase_date,first_purchase.cohort,purchase_date.cohort,customer_type.period,returned
0,25890929042,266783,2011-09-23,1,2,4,1321,554.82,5838.82,e-Shop,yes,no,Footwear,Mens,1,2014-12-02,394.00,23,9,2011,2011_09,0,06_Friday,38,09_Sep,6,month.week4,2011-09-23,2011_09,2011_09,new,no
1,98477711300,266783,2012-10-21,4,1,3,93,29.30,308.30,TeleShop,no,no,Clothing,Mens,1,2014-12-02,122.00,21,10,2012,2012_10,0,01_Sunday,42,10_Oct,1,month.week3,2011-09-23,2011_09,2012_10,existing,no


## Data Audit

In [ ]:
df_customer.columns = [c.lower() for c in df_customer.columns]
df_customer.shape
mt.check_unique_no(df_customer, ['customer_id'])
mt.missing_data_table(df_customer)
df_customer.head()

In [ ]:
df_transactions.rename(columns={'transaction_id':'trans_id', 'cust_id':'customer_id', 'tran_date':'trans_date'}, inplace=True)
df_transactions.columns = [c.lower() for c in df_transactions.columns]
df_transactions.shape
mt.check_unique_no(df_transactions, ['customer_id', 'trans_id'])
mt.missing_data_table(df_transactions)
df_transactions.head()
df_transactions.describe()

In [ ]:
df_transactions['trans_id'].value_counts(dropna=False).head()

In [ ]:
df_transactions[df_transactions['trans_id'] == 426787191]

In [ ]:
df_transactions[df_transactions['trans_id'] == 4170892941]

In [ ]:
df_transactions[df_transactions['trans_id'] == 3130889793]

In [ ]:
df_transactions['trans_id'].value_counts(dropna=False)[df_transactions['trans_id'].value_counts(dropna=False).values > 1].index.nunique()

In [ ]:
df_transactions[df_transactions['qty'] < 0].shape
df_transactions[df_transactions['qty'] < 0].head()
df_transactions[df_transactions['qty'] < 0]['store_type'].value_counts(dropna=False)

In [ ]:
df_transactions[df_transactions['qty'] > 0].shape
df_transactions[df_transactions['qty'] > 0].head()
df_transactions[df_transactions['qty'] > 0]['store_type'].value_counts(dropna=False)

In [ ]:
df_products.rename(columns={'prod_sub_cat_code':'prod_subcat_code'}, inplace=True)
df_products['counter'] = 1
mt.missing_data_table(df_products)
df_products.head()

In [ ]:
prod_cat_dict = dict(zip(df_products['prod_cat_code'],df_products['prod_cat']))
prod_subcat_dict = dict(zip(df_products['prod_subcat_code'],df_products['prod_subcat']))

In [ ]:
df_products.groupby(['prod_cat'])['prod_subcat'].value_counts(dropna=False)

In [ ]:
df_products['prod_cat'].value_counts(dropna=False)

In [ ]:
df_products['prod_subcat'].value_counts(dropna=False)

### Transactions Data

In [ ]:
converted_customers = df_transactions['customer_id'].unique().tolist()

In [ ]:
df_transactions['trans_date'] = pd.to_datetime(df_transactions['trans_date'])
df_transactions['trans_date'].sort_values().min()
df_transactions['trans_date'].sort_values().max()

In [ ]:
df_transactions = pd.merge(df_transactions, df_products, on=['prod_cat_code', 'prod_subcat_code'], how='left')
df_transactions.head()

In [ ]:
# df_transactions['qty'] = np.where((df_transactions['qty'] < 0), (-1*df_transactions['qty']), df_transactions['qty'])
# df_transactions['rate'] = np.where((df_transactions['rate'] < 0), (-1*df_transactions['rate']), df_transactions['rate'])
# df_transactions['total_amt'] = np.where((df_transactions['total_amt'] < 0), (-1*df_transactions['total_amt']), df_transactions['total_amt'])
# df_transactions.head()

In [ ]:
df_transactions['store_type'].value_counts(dropna=False)

In [ ]:
df_transactions = df_transactions.sort_values(['customer_id','trans_date'])

In [ ]:
df_transactions['assessment_date'] = df_transactions['trans_date'].sort_values().max()

In [ ]:
df_transactions['duration'] = df_transactions.groupby(['customer_id'])['trans_date'].transform(pd.Series.diff).shift(-1)
df_transactions['duration'] = df_transactions.apply(lambda r: r['assessment_date'] - r['trans_date'] if pd.isnull(r['duration']) else r['duration'], axis=1)
df_transactions['duration'] = (df_transactions['duration']/np.timedelta64(1, 'D'))

In [ ]:
df_transactions.head(7)

In [ ]:
df_trans_overall = df_transactions.groupby(['customer_id'], as_index=False).agg({'trans_date':['first', 'last', pd.Series.nunique],
                                                                                 'duration': ['min', 'max', 'mean', 'last'],
                                                        'trans_id':'count',
                                                        'qty':['min', 'max', 'sum'],
                                                       'tax':'sum',
                                                       'total_amt':'sum' }) #, lambda x: stats.mode(x)[0][0]]|, 'prod_cat':[pd.Series.nunique],
#                                                          'prod_subcat':[pd.Series.nunique],
#                                                        'store_type':[pd.Series.nunique]

In [ ]:
df_trans_overall.shape
df_trans_overall.columns = [".".join(x).strip('.') for x in df_trans_overall.columns.ravel()] 
df_trans_overall.rename(columns={'trans_date.first':'conversion_date', 'trans_date.last':'last_purchase_date',
                                'trans_date.nunique':'number_of_purchase_days'}, inplace=True)
mt.check_unique_no(df_trans_overall, ['customer_id'])
df_trans_overall.head()

In [ ]:
df_trans_overall['assessment_date'] = df_transactions['trans_date'].sort_values().max()

In [ ]:
df_trans_overall['customer_age.days'] = (df_trans_overall['assessment_date'] - df_trans_overall['conversion_date'])/np.timedelta64(1,'D')

In [ ]:
df_trans_overall['customer_age.years'] = (df_trans_overall['assessment_date'] - df_trans_overall['conversion_date'])/np.timedelta64(1,'Y')

In [ ]:
age_bins =  [0, 1, 2, 3, 4]
labels = ['_<01','01_02', '02_03', '03_04']
df_trans_overall['customer_age.years.group'] = pd.cut(df_trans_overall['customer_age.years'], age_bins, labels = labels,include_lowest = True)

In [ ]:
# def customer_profile(_df, unique_id, value, featurelist):
#     df_list = []
#     for feature in featurelist:
#         _df[feature] = _df[feature].astype(str)
#         _df_temp = pd.crosstab(_df[unique_id], _df[feature], values=_df[value], 
#                                aggfunc=['count','sum', 'mean'], dropna=False).fillna(0).reset_index()
#         _df_temp.columns = [".".join(x).strip('.') for x in _df_temp.columns.ravel()] 
#         df_list.append(_df_temp)
#         dfs = [df.set_index(unique_id) for df in df_list]
#         df = pd.concat(dfs, axis=1)
#         df.reset_index(inplace=True)
#     return df

In [ ]:
# df_product_cat = customer_profile(df_transactions, 'customer_id', 'total_amt', ['prod_cat'])
# df_product_cat.head()

In [ ]:
# df_product_subcat = customer_profile(df_transactions, 'customer_id', 'total_amt', ['prod_subcat'])
# df_product_subcat.head()

In [ ]:
# df_stores = customer_profile(df_transactions, 'customer_id', 'total_amt', ['store_type'])
# df_stores.head()

In [ ]:
# df_trans_overall = pd.merge(df_trans_overall, df_stores, on=['customer_id'], how='left')
# df_trans_overall.head()

In [ ]:
# df_trans_overall['Flagship_store_spend.prop'] = df_trans_overall['sum.Flagship store']/df_trans_overall['total_amt.sum']
# df_trans_overall['MBR_spend.prop'] = df_trans_overall['sum.MBR']/df_trans_overall['total_amt.sum']
# df_trans_overall['TeleShop_spend.prop'] = df_trans_overall['sum.TeleShop']/df_trans_overall['total_amt.sum']
# df_trans_overall['e-Shop.prop'] = df_trans_overall['sum.e-Shop']/df_trans_overall['total_amt.sum']

In [ ]:
# df_trans_overall = pd.merge(df_trans_overall, df_product_cat, on=['customer_id'], how='left')
# df_trans_overall.head()

In [ ]:
# df_trans_overall['Bags.prop'] = df_trans_overall['sum.Bags']/df_trans_overall['total_amt.sum']
# df_trans_overall['Books.prop'] = df_trans_overall['sum.Books']/df_trans_overall['total_amt.sum']
# df_trans_overall['Clothing.prop'] = df_trans_overall['sum.Clothing']/df_trans_overall['total_amt.sum']
# df_trans_overall['Electronics.prop'] = df_trans_overall['sum.Electronics']/df_trans_overall['total_amt.sum']
# df_trans_overall['Footwear.prop'] = df_trans_overall['sum.Footwear']/df_trans_overall['total_amt.sum']
# df_trans_overall['Home and kitchen.prop'] = df_trans_overall['sum.Home and kitchen']/df_trans_overall['total_amt.sum']

In [ ]:
# df_trans_overall = pd.merge(df_trans_overall, df_product_subcat, on=['customer_id'], how='left')
# df_trans_overall.head()

In [ ]:
# df_trans_overall['Academic.prop'] = df_trans_overall['sum.Academic']/df_trans_overall['total_amt.sum']
# df_trans_overall['Audio and video.prop'] = df_trans_overall['sum.Audio and video']/df_trans_overall['total_amt.sum']
# df_trans_overall['Bath.prop'] = df_trans_overall['sum.Bath']/df_trans_overall['total_amt.sum']
# df_trans_overall['Cameras.prop'] = df_trans_overall['sum.Cameras']/df_trans_overall['total_amt.sum']
# df_trans_overall['Children.prop'] = df_trans_overall['sum.Children']/df_trans_overall['total_amt.sum']
# df_trans_overall['Comics.prop'] = df_trans_overall['sum.Comics']/df_trans_overall['total_amt.sum']
# df_trans_overall['Computers.prop'] = df_trans_overall['sum.Computers']/df_trans_overall['total_amt.sum']
# df_trans_overall['DIY.prop'] = df_trans_overall['sum.DIY']/df_trans_overall['total_amt.sum']
# df_trans_overall['Fiction.prop'] = df_trans_overall['sum.Fiction']/df_trans_overall['total_amt.sum']
# df_trans_overall['Furnishing.prop'] = df_trans_overall['sum.Furnishing']/df_trans_overall['total_amt.sum']
# df_trans_overall['Kids.prop'] = df_trans_overall['sum.Kids']/df_trans_overall['total_amt.sum']
# df_trans_overall['Kitchen.prop'] = df_trans_overall['sum.Kitchen']/df_trans_overall['total_amt.sum']
# df_trans_overall['Mens.prop'] = df_trans_overall['sum.Mens']/df_trans_overall['total_amt.sum']
# df_trans_overall['Mobiles.prop'] = df_trans_overall['sum.Mobiles']/df_trans_overall['total_amt.sum']
# df_trans_overall['Non-Fiction.prop'] = df_trans_overall['sum.Non-Fiction']/df_trans_overall['total_amt.sum']
# df_trans_overall['Personal Appliances.prop'] = df_trans_overall['sum.Personal Appliances']/df_trans_overall['total_amt.sum']
# df_trans_overall['Tools.prop'] = df_trans_overall['sum.Tools']/df_trans_overall['total_amt.sum']
# df_trans_overall['Women.prop'] = df_trans_overall['sum.Women']/df_trans_overall['total_amt.sum']

In [ ]:
df_trans_overall.head()

### Customer Data

In [ ]:
df_customer['customer_status'] = np.where((df_customer['customer_id'].isin(converted_customers)), 'converted', 'voluntarily churned - not converted')
df_customer['customer_status'].value_counts(dropna=False)

In [ ]:
df_customer['dob'] = pd.to_datetime(df_customer['dob'])
df_customer['dob'].sort_values().min()
df_customer['dob'].sort_values().max()

In [ ]:
df_customer.tail()

In [ ]:
df_transactions['trans_date.day'] = df_transactions['trans_date'].dt.day
df_transactions['trans_date.month_num'] = df_transactions['trans_date'].dt.month.map("{:02}".format)
df_transactions['trans_date.year'] = df_transactions['trans_date'].dt.year
df_transactions['trans_date.year_month'] = df_transactions['trans_date'].dt.year.map(str) + '_' + df_transactions['trans_date'].dt.month.map("{:02}".format)
df_transactions['trans_date.hour'] = df_transactions['trans_date'].dt.hour
df_transactions['trans_date.weekday'] = df_transactions['trans_date'].dt.day_name()
df_transactions['trans_date.week_of_year'] = df_transactions['trans_date'].dt.week.map("{:02}".format)

In [ ]:
df_transactions['trans_date.weekday'].head()

In [ ]:
month_name = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
month_num = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
month_dict = dict(zip(month_num, month_name))

for k, v in month_dict.items():
    mask = df_transactions['trans_date.month_num'].str.contains(k, case=True)

    df_transactions.loc[mask,'trans_date.month'] = v

In [ ]:
df_transactions['trans_date.month'] = df_transactions['trans_date.month_num'] + '_' + df_transactions['trans_date.month'].map(str)

In [ ]:
df_transactions['trans_date.month'].value_counts(dropna=False)

In [ ]:
weekday_name = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
weekday_num = ['01', '02', '03', '04', '05', '06', '07']
weekday_dict = dict(zip(weekday_name, weekday_num))

for k, v in weekday_dict.items():
    mask = df_transactions['trans_date.weekday'].str.contains(k, case=True)

    df_transactions.loc[mask, 'trans_date.weekday_num'] = v

In [ ]:
def bin_week(x):
    if ( x>= 1) and (x <= 7):
        return 'month.week1'

    elif (x >= 8) and (x <= 14):
        return 'month.week2'

    elif (x >= 15) and (x <= 21):
        return 'month.week3'

    else:
        return 'month.week4'

In [ ]:
df_transactions['trans_date.week_of_month'] = df_transactions['trans_date'].dt.day.apply(bin_week)

In [ ]:
df_transactions['trans_date.weekday'] = df_transactions['trans_date.weekday_num'] + '_' + df_transactions['trans_date.weekday'].map(str)

In [ ]:
df_transactions['trans_date.weekday'].value_counts(dropna=False)

In [ ]:
# df_transactions['trans_date.hour'].describe()
# df_transactions['trans_date.time_of_day']

In [ ]:
df_transactions = df_transactions.sort_values(['customer_id', 'trans_date'])

In [ ]:
df_first_purchase_date = df_transactions.groupby(['customer_id']).agg({'trans_date':'first'})

In [ ]:
df_first_purchase_date.rename(columns={'trans_date':'first_purchase_date'}, inplace=True)

In [ ]:
df_first_purchase_date.shape

In [ ]:
df_transactions = pd.merge(df_transactions, df_first_purchase_date, on='customer_id', how='left')

In [ ]:
df_transactions['first_purchase.cohort'] = df_transactions['first_purchase_date'].dt.year.map(str) + "_" + df_transactions['first_purchase_date'].dt.month.map("{:02}".format)

In [ ]:
df_transactions['purchase_date.cohort'] = df_transactions['trans_date'].dt.year.map(str) + "_" + df_transactions['trans_date'].dt.month.map("{:02}".format)

In [ ]:
df_earliest_purchase_date = df_first_purchase_date.copy()
df_earliest_purchase_date.rename(columns={'first_purchase_date':'trans_date'}, inplace=True)

In [ ]:
df_earliest_purchase_date['customer_type.period'] = 'new'

In [ ]:
df_transactions = pd.merge(df_transactions, df_earliest_purchase_date, on=['customer_id', 'trans_date'], how='left')

In [ ]:
df_transactions['customer_type.period']  = np.where(df_transactions['customer_type.period'].isnull(), 'existing', df_transactions['customer_type.period'])

In [ ]:
df_transactions['qty_negative'] = np.where((df_transactions['qty'] < 0), 'yes', 'no')

In [ ]:
trans_id_returned = df_transactions[df_transactions['qty_negative'] == 'yes']['trans_id'].unique().tolist()

In [ ]:
df_transactions['returned'] = np.where((df_transactions['trans_id'].isin(trans_id_returned)), 'yes', 'no')

In [ ]:
df_transactions.drop(['qty_negative'], axis=1, inplace=True)

In [ ]:
df_transactions.head()

In [ ]:
df_transactions.tail()

In [ ]:
df_transactions[df_transactions['customer_id'] == 268624]

### Master File

In [ ]:
df_master = pd.merge(df_customer, df_trans_overall,  on=['customer_id'], how='left')
# df_master['assessment_date'] = df_transactions['trans_date'].sort_values().max()
mt.check_unique_no(df_master, ['customer_id'])
df_master.head()

In [ ]:
df_master['customer_status'].value_counts(dropna=False)

In [ ]:
df_master['conversion_date.month_num'] = df_master['conversion_date'].dt.month.map("{:02}".format)

In [ ]:
df_master['conversion_date.month_num'].value_counts(dropna=False)

In [ ]:
# df_master['conversion_date.month_num'] = df_master['conversion_date.month_num'].astype('O').astype('int64')

In [ ]:
month_name = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
month_num = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
month_dict = dict(zip(month_num, month_name))

for k, v in month_dict.items():
    mask = df_master['conversion_date.month_num'].str.contains(k, case=True)

    df_master.loc[mask,'conversion_date.month'] = v

In [ ]:
df_master['conversion_date.month'] = df_master['conversion_date.month_num'] + '_' + df_master['conversion_date.month'].map(str)

In [ ]:
df_master['conversion_date.month'].value_counts(dropna=False)

In [ ]:
df_master.head(1)

In [ ]:
df_master['biological_age.actual'] = (df_master['assessment_date'] - df_master['dob'])/np.timedelta64(1, 'Y')

In [ ]:
df_master['biological_age.actual'].describe()

In [ ]:
df_master['biological_age'] = np.round(df_master['biological_age.actual'])

In [ ]:
df_master['biological_age'].describe()

In [ ]:
age_bins =  [21, 25, 30, 35, 40, 45]
labels = ['22_25','25_30', '30_35', '35_40', '40_45']
df_master['biological_age.group'] = pd.cut(df_master['biological_age'], age_bins, labels = labels,include_lowest = True)

In [ ]:
df_master['number_of_purchase_days'].describe()

In [ ]:
df_master['number_of_purchase_days'] =df_master['number_of_purchase_days'].replace(np.nan, 0)

In [ ]:
df_master['repeat_purchaser'] = np.where((df_master['number_of_purchase_days'] > 1), 'yes',
                                      np.where((df_master['number_of_purchase_days']== 0),'never purchased',
                                        'no'))

In [ ]:
df_master[df_master['customer_id'] == 266783]

In [ ]:
df_master[df_master['customer_age.years'] == df_master['customer_age.years'].min()]

In [ ]:
df_master['1_day_amt.avg'] = (df_master['total_amt.sum']/df_master['customer_age.days']) * 1
df_master['7_day_amt.avg'] = (df_master['total_amt.sum']/df_master['customer_age.days']) * 7
df_master['30_day_amt.avg'] = (df_master['total_amt.sum']/df_master['customer_age.days']) * 30

In [ ]:
df_master['1_day_num.avg'] = (df_master['qty.sum']/df_master['customer_age.days']) * 1
df_master['7_day_num.avg'] = (df_master['qty.sum']/df_master['customer_age.days']) * 7
df_master['30_day_num.avg'] = (df_master['qty.sum']/df_master['customer_age.days']) * 30

In [ ]:
df_master[df_master['repeat_purchaser'] == 'yes'].head()

In [ ]:
df_master['repeat_purchaser'].value_counts(dropna=False)

In [ ]:
df_master['qty.sum'].sum()

In [ ]:
df_master['total_amt.sum'].sum()

In [ ]:
df_master['total_amt.sum'].sum() - df_master['tax.sum'].sum() 

In [ ]:
df_transactions.to_csv(filepaths.interim_transactions_data, index=False)
df_master.to_csv(filepaths.master_file_data, index=False)